# Aircraft Surface Positioning Pipeline Demo

- Dataset: Tufts univerity turf field
- Pose estimations: Generated through COLMAP (Schonberger and Frahm, 2016)



NOTES:
- All 'Visualizer' steps are for visualization purposes only, can be commented out
- Recommended to start with 5 local images, as solution with 10 images begins to drift (see paper for details)

In [1]:
from groundNAV_agent import *
from pathlib import Path

/home/daniel-choate/miniconda3/envs/py39/lib/python3.9/site-packages/dash/dash.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, parse_version


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Input files: 
- SfM solution (3)
    - Images
    - Cameras
    - 3D Cloud
- Local Images for registration (10)
- Satellite reference image (1)

In [2]:
# SfM files 
images_colm = '../sample_data/SfM_soln/images.txt'
cameras_colm = '../sample_data/SfM_soln/cameras.txt'
pts3d_colm = '../sample_data/SfM_soln/points3D_f.txt'

# Local images - folder
# im_local = '../sample_data/local_imgs_10' # With 10 local images 
im_local = '../sample_data/local_imgs_5' # With 5 local images

# Satellite reference image
sat_ref = '../sample_data/TurfSat.jpg'

In [3]:
# Create class 
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, im_local, sat_ref)

___________________________________
### Visualizer (1)
- Initial pose estimation and sparse point cloud

In [4]:
# # Visualize the sparse scene and camera poses generated through SfM 

# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name = "Pose estimations and sparse cloud: SfM COORDINATES (not absolute)")

# gnav.pose_scene_visualization(vis)

__________________________________________

## Set Reference Frame 

In [5]:
# Set reference frame - transform to ground-parallel coordinate frame 
tform_ref_frame = gnav.set_ref_frame(gnav.pts_gnd_idx)
tform_ref_frame_pts = gnav.inv_homog_transform(tform_ref_frame)
print("\nReference frame transformation\n", tform_ref_frame_pts)

# Transfer all points to new coordinate system
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(gnav.scene_pts, gnav.origin_w, tform_ref_frame_pts)


Reference frame transformation
 [[-1.55069060e-03  9.81197008e-01  1.93002661e-01 -1.21025836e-01]
 [-1.42845166e-01 -1.91240997e-01  9.71093270e-01  1.86102525e+00]
 [ 9.89743833e-01 -2.60636319e-02  1.40455805e-01  7.28134156e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


____________________________________
### Visualizer (2)

- Sparse point cloud in NEW (ground-referenced) frame

In [6]:
# # Visualize the sparse scene and camera poses generated through SfM 

# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name = "Sparse cloud: ground-reference coords")

# gnav.pose_scene_visualization_ref(vis, scene_pts_ref)

__________________________________________________________

## Image Mosaic Formulation

In [7]:
# Import mosaic parameters - arbitrarily obtained, can be modified
mosaic_params = np.load('../sample_data/GP_sections/mosaic_params.npy')
# Grab specified image points from local images 
gnav.grab_image_pts_tot(mosaic_params)

____________________________________________
### Visualizer (3)

- Ground plane sections from 2D image to be used for mosaic

In [8]:
# plt.figure(figsize=(15,8))
# gnav.plot_gnd_pts()

_________________________________________________________

## Image mosaic formulation stages 
1. Create unit vectors in camera coordinates
2. Grab transformation matrix (camera to world (SfM) coords)
3. Transform to world (SfM) coords
4. Projection on ground plane
5. Transform to ground-referenced plane

In [9]:
# Generate projection of image sections 
for i in range(len(gnav.images_dict)):
    # STEP 1: Unit vectors in camera coords 
    pts_vec_c, pts_rgb_gnd = gnav.unit_vec_c(i)
    gnav.im_mosaic[i] = {'rgbc': pts_rgb_gnd}

    # STEP 2: Transformation matrix moves from camera coords to world coords
    id = gnav.im_ids[i]
    homog_w2c, homog_c2w = gnav.get_pose_id(id,i)
    # print('Homogeneous transformation from world to camera \n', homog_c2w)
    # print('\n Homogeneous transformation from camera to world \n', homog_w2c)

    # STEP 3: Transform to world coords
    origin_c, pts_loc_w, pts_vec_w = gnav.unit_vec_tform(pts_vec_c, gnav.origin_w, homog_c2w)
    # print('\n New camera frame origin = ', origin_c)
    
    # STEP 4: Get new points 
    ranges, new_pts_w = gnav.pt_range(pts_vec_w, homog_c2w, origin_c, i)
    # print('\nNew Points \n', new_pts_w)

    # STEP 5: Transfer points to reference frame
    __, new_pts_r, pts_vec_r = gnav.unit_vec_tform(new_pts_w, gnav.origin_w, tform_ref_frame_pts)

    # Convert points to grayscale 
    gray_c = gnav.conv_to_gray(gnav.im_mosaic[i]['rgbc'],i)
    # print(gray_c)

    # Put new points and grayscale colors in image mosaic
    gnav.im_mosaic[i]['pts'] = new_pts_r
    gnav.im_mosaic[i]['color_g'] = gray_c
    
    print("\nDone image ", i)


Done image  0

Done image  1

Done image  2

Done image  3

Done image  4


_____________________________________
### Visualizer (4)

- Image mosaic formulation

In [10]:
# # Create visualization
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Image mosaic formulation")

# gnav.mosaic_visualization(vis)

________________________________________

## Implement initial guess

- Initial guess to be fed into least squares process
- Must be good enough to allow convergence

### 4 DOF Solution

The state equation includes the following variables:

- **x**: position along the x-axis  
- **y**: position along the y-axis  
- **$\boldsymbol{\theta}$**: heading (yaw angle)
- **s**: scale factor

**State vector:**  
$x = [x,\ y,\ \theta, s]$

In [11]:
# Initial guess parameters 
# 5 image solution
scale, yaw, x_g, y_g = 80, np.deg2rad(140), -52, 20

tform_guess = gnav.tform_create(x_g,y_g,0,0,0,yaw)
gnav.best_guess_tform = tform_guess
gnav.best_guess_scale = scale

# Implement
gnav.implement_guess(gnav.best_guess_tform, gnav.best_guess_scale)

__________________________________________________
### Visualizer (5)

- Mosaic with reference satellite map (with initial guess)

In [12]:
# Create visualization
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Image mosaic with reference map (initial guess)")

gnav.mosaic_w_ref_visualization(vis)

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 73
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 73
pci id for fd 73: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


_________________________________________

## Least squares optimization

### SSD measurements
- A sum of squared differences (SSD) of intensities is used as a residual measurement for the least squares optimization 
- Based on pixel shifts of an nxn grid (see paper)
- Default n = 5: balancing runtime and accuracy

### Convergence
- See paper for jacobian construction details
- Note: different initial guesses will require varied amounts of iterations to reach convergence

In [13]:
# Initialize pixel search threshold and iterations
n = 10 # Pixel search threshold
iterations = 1

# Set initial guess parameters
s, theta, tp, tq = scale, yaw, x_g, y_g
gnav.params_best_guess = np.array([scale, theta, tp, tq]).reshape(4,1)

# Results in parameters best guess 
params_best_guess = gnav.mapmatch_lsquares(n, iterations, gnav.params_best_guess)

Number of points used for image 0:  (1007,)
Number of points used for image 1:  (637,)
Number of points used for image 2:  (2821,)
Number of points used for image 3:  (719,)
Number of points used for image 4:  (4748,)
Here is y_i for iteration 0:
 [[-2.]
 [ 5.]
 [-7.]
 [-4.]
 [-5.]
 [ 3.]
 [-4.]
 [ 4.]
 [ 1.]
 [ 3.]]

JACOBIAN
 [[ 7.86600947e+01 -1.90098746e+03  1.00000000e+00  0.00000000e+00]
 [ 2.37623433e+01  6.29280757e+03  0.00000000e+00  1.00000000e+00]
 [-6.22606819e+01  3.81070520e+02  1.00000000e+00  0.00000000e+00]
 [-4.76338151e+00 -4.98085455e+03  0.00000000e+00  1.00000000e+00]
 [-2.01920365e+00 -6.89483664e+03  1.00000000e+00  0.00000000e+00]
 [ 8.61854580e+01 -1.61536292e+02  0.00000000e+00  1.00000000e+00]
 [ 4.07330064e+01 -5.12409307e+03  1.00000000e+00  0.00000000e+00]
 [ 6.40511634e+01  3.25864051e+03  0.00000000e+00  1.00000000e+00]
 [ 2.56187829e+01  8.86688749e+03  1.00000000e+00  0.00000000e+00]
 [-1.10836094e+02  2.04950263e+03  0.00000000e+00  1.00000000e+00]]

In [13]:
# # Start by testing a single SSD

# n = 10
# # for imnum in range(len(images)):
# # Just for the first image for now 
# for imnum in range(0, 1, 1):
#     print(imnum)
#     ssds = gnav.ssd_nxn(n, imnum)
#     gnav.ssds_curr[imnum] = ssds
# # print("Done, most recent SSDs are\n", ssds)

In [14]:
# # PLOT SSDS
# # Plot and visualize SSD values (was originally used in LSquares Loop 

# # Create a 5x5 grid of x and y coordinates
# x = np.linspace(-n, n, 2*n+1)
# y = np.linspace(-n, n, 2*n+1)
# Y, X = np.meshgrid(x, y)
# # print(ssds)

# # print(x)
# # print(X)
# # print(Y)
# i = 0
# # Best shift
# idrow, idcol = np.unravel_index(np.argmin(gnav.ssds_curr[i]), ssds.shape)
# print(idrow, idcol)
# shiftx_min = idrow-n
# shifty_min = idcol-n
# # print("BEST SHIFT = ", shiftx_min, shifty_min)
# # print("BEST SSD = ", SSDS_CURR[i][idrow, idcol])

# # PLOTTING A SINGLE VECTOR FIELD FOR SSD
# # Create the figure and 3D axis
# %matplotlib qt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Plot the 3D vectors
# ax.quiver(X, Y, np.zeros_like(gnav.ssds_curr[i]), np.zeros_like(gnav.ssds_curr[i]), np.zeros_like(gnav.ssds_curr[i]), (gnav.ssds_curr[i]/1000)**2, arrow_length_ratio=0.1)

# # Set axis limits
# ax.set_xlim([-11, 11])  # X axis range
# ax.set_ylim([-11, 11])  # Y axis range
# ax.set_zlim([0, (np.max(gnav.ssds_curr[i]) / 1000)**2])  # Z axis range, adjust based on your data

# # Labels and title
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('Pixel Correction Vector Field')

In [15]:
# iterations = 1
# n = 10
# extend = 0
    
# for iter_idx in range(iterations):
#     # Loop through full set of images
#     for imnum in range(len(gnav.images_dict)):
#         ssds = gnav.ssd_nxn(n, imnum)
#         gnav.ssds_curr[imnum] = ssds
        
#     # print("Most recent SSDs\n", ssds)
    

    
    
#     # Create a vector from the original position to the minimum SSD location 
#     cor_vecs = np.zeros((len(gnav.images_dict), 2))
#     base_vec = np.zeros((len(gnav.images_dict), 2))
#     for im_cv in range(len(gnav.images_dict)):
#     # for imnum in range(1):
#         # Grab SSDs - get id of minimum
#         ssds = gnav.ssds_curr[im_cv]
#         # print(f"\nSSDS for image {i}\n", ssds)
#         idrow, idcol = np.unravel_index(np.argmin(ssds), ssds.shape)
#         # print("\nidrow, idcol\n", idrow, idcol)
#         # Define best shift vector 
#         shiftx_min = idrow-n
#         shifty_min = idcol-n
#         # CHECK IF MIN SSD IS ON EDGE
#         if shiftx_min == n or shifty_min == n:
#             print(f"\n Need to extend search of image {im_cv}\n")
#             print(f"\n Current shift vector = {shiftx_min, shifty_min}\n")
#             ssds = gnav.ssd_nxn(n+extend, im_cv)
#             gnav.ssds_curr[im_cv] = ssds
#             idrow, idcol = np.unravel_index(np.argmin(ssds), ssds.shape)
#             shiftx_min = idrow-(n+extend)
#             shifty_min = idcol-(n+extend)
#             print(f"\n New shift vector = {shiftx_min, shifty_min}\n")
            
        
#         # print("\nBEST SHIFT VECTOR = ", shiftx_min, shifty_min)
#         # print(f"\nBEST SSD for image {im_cv} = ", ssds[idrow, idcol])
#         cor_vecs[im_cv] = shiftx_min, shifty_min
#         # Get mean of satellite points for base of x and y
#         sat_pts, __ = gnav.get_inside_sat_pts(im_cv, 0,0)#shiftx_min, shifty_min)
#         # print("\nInside satellite points\n", sat_pts)
#         basex, basey = np.mean(sat_pts[:,0]), np.mean(sat_pts[:,1])
#         # print("\nBase of x and y = ", basex, basey)
#         base_vec[im_cv] = basex, basey
    
#     # print("\nCorrection vectors:\n", cor_vecs)
#     # print("\nBase of correction vectors: \n", base_vec)
    
#     points_b = np.hstack((base_vec, np.zeros((len(gnav.images_dict), 1))))
#     points_e = points_b + np.hstack((cor_vecs, np.zeros((len(gnav.images_dict),1))))
#     points = np.vstack((points_b, points_e))
#     # print("\nBeginning of points: \n", points_b)
#     # print("\nEnd of points: \n",points_e)
#     # print("\nAll points: \n",points)
#     # lines = []
#     # for lin in range(len(images)):
#     #     lines.append([lin,lin+len(images)])
#     #     # print(lin, lin+len(images))
    
#     # print("\nLines for OPEN3d:\n",lines)
    
#     # Delta y term
#     # print(cor_vecs.reshape(-1,1))
#     yi = cor_vecs.reshape(-1,1)
#     print("\nYi\n", yi)
    
    
    
    
    
#     # Form jacobian for each image
#     J = np.zeros((2*len(gnav.images_dict),4))
#     theta = params[1][0]
#     s = params[0][0]
#     # print(J)
#     for i_m in range(len(gnav.images_dict)):
#         xpi = np.mean(gnav.im_pts_best_guess[i_m]['pts'][:,0])
#         xqi = np.mean(gnav.im_pts_best_guess[i_m]['pts'][:,1])
#         # print(xpi, xqi)
#         # Jacobian values (2x4)
#         j11 = np.cos(theta)*xpi - np.sin(theta)*xqi
#         j21 = np.sin(theta)*xpi + np.cos(theta)*xqi
#         j12 = -params[0][0]*(np.sin(theta)*xpi + np.cos(theta)*xqi)
#         j22 = params[0][0]*(np.cos(theta)*xpi - np.sin(theta)*xqi)
#         j13, j23, j14, j24 = 1, 0, 0, 1
#         # print(j13)
    
#         # Construct Jacobian 
#         J_upper = np.hstack((j11, j12, j13, j14))  # First row block (y_p terms)
#         # print("\nJ Upper\n", J_upper)
#         J_lower = np.hstack((j21, j22, j23, j24))  # Second row block (y_q terms)
#         # print("\nJ Lower\n", J_lower)
#         j = np.vstack((J_upper, J_lower))  # Shape (2N, 4)
#         # print("\nJacobian\n", j)
#         J[2*(i_m):2*(i_m)+2, :] = j
    
#     print("\nJACOBIAN\n", J)
#     # print("\nJacobian shape: ", J.shape)
    
#     # Least squares process
#     JTJi = np.linalg.inv(J.T@J)
#     Dalpha = JTJi@J.T@yi
#     print("\nDelta Alpha:\n", Dalpha)
    
    
    
#     params += Dalpha
#     print("\nUpdated Params: scale, theta, tq, tp\n", params)
    
    
    
#     # Apply change
#     tform_mat = gnav.tform_create(params[2][0], params[3][0], 0, 0, 0, params[1][0]) # x,y,z,roll,pitch,yaw
#     print("\nTransformation matrix\n", tform_mat)
    
    
#     # for i_c in range(len(images)):
#     # # Just for the first image for now
#     # # for i in range(1):
#     #     loc_im_pts = gnav.im_mosaic[i_c]['pts'].copy()
#     #     # print(loc_im_pts)
#     #     loc_im_pts[:, :2] *= params[0][0].astype(np.float64)
#     #     # Get new points 
#     #     __, loc_im_pts_NEW, __ = gnav.unit_vec_tform(loc_im_pts, gnav.origin_w, tform_mat)
#     #     gnav.im_pts_best_guess[i_c]['pts'] = loc_im_pts_NEW
#     #     # gnav.im_pts_best_guess[i]['tree'] = cKDTree(loc_im_pts_guess) # UNECESSARY 
    
#     #     # print("\nDone image ", i)